## Set up and imports
---

In [ ]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import *
import matplotlib.pyplot as plt
import numpy as np
import json
import os

from parse import read_file

In [ ]:
file_path = "data/final_data/low_os/d_3_specialized_tasks.alb"

(
    OperationsLength,
    cycle_time,
    order_strength,
    op_times,
    PrecRelations,
    number_of_worker_categories,
    worker_availability,
    worker_modifiers,
    StationsLength,
    station_capacity,
    worker_amount_modifiers,
    worker_bounds,
) = read_file(file_path)

## SALBP-2 model formulation
-----------------------------------------

### Objective functions:

 1) The goal of the first objective is to minimize the cycle time:   $ \hspace{31mm} min \quad z = Cycle\_Time $

 2) The goal of the second objective is minimize the difference in time each worker is occupied during a cycle:   
 
 $ \hspace{10mm} min \quad z = \sum_{worker = 1}^{Worker} (Max\_Workhours\_Var - Worker\_actual\_time\_Var_{worker})$
 
 
 <!---$ \hspace{0mm} min \quad z = \sum_{worker_1 = 1}^{Worker} max_{worker_2 \in Worker}(\sum_{op=1}^{Operations} Worker\_at\_Op_{worker_2,op} \cdot Actual\_processing\_time\_per\_Op_{op} ) - \sum_{op=1}^{Operations} Worker\_at\_Op_{worker_1,op} \cdot Actual\_processing\_time\_per\_Op_{op}  $--->


### Constraints:

1) Assignment of  every  operation  only once: $\hspace{25mm} \sum_{k=1}^MOperation\_at\_Station_{ik}=1  \hspace{39mm} \forall i=1,...,N $ 

2) Complying with the precedence relations: $\hspace{15mm} \sum_{k=1}^MOperation\_at\_Station_{ik}\cdot k \leq \sum_{k=1}^MOperation\_at\_Station_{jk}\cdot k \hspace{3mm} 
\forall(i,j) \in P$

3) Complying with the cycle time: $\hspace{33mm} \sum_{i=1}^NOperation\_at\_Station_{ik} \cdot T_i \leq Cycle\_Time \hspace{33mm} \forall k=1,...,M $

4) Binary decision variable: $\hspace{40mm} Operation\_at\_Station_{ik} \in \{0,1\} \hspace{41mm} \forall i=1,...,N;k=1,...,M $

5) Non-negativity constraint: $\hspace{62mm} Cycle\_Time \geq0$

6) Workers' ops only at one station: 

$ \hspace{10mm} \sum_{station=1}^{Stations} Worker\_at\_Station_{worker,station}\leq 1\hspace{10mm}\forall worker\in Workers$

7) Operation worker bounds fulfilled: 

$ \hspace{10mm} \sum_{worker=1}^{Worker} Worker\_At\_Op_{worker,op} \leq Worker\_Bounds\_per\_Op_{op,ub}$ 

$ \hspace{10mm} \forall op\in Operations$

$ \hspace{10mm} \sum_{worker=1}^{Worker} Worker\_At\_Op_{worker,op} \geq Worker\_Bounds\_per\_Op_{op,lb}$

$ \hspace{10mm} \forall op\in Operations$

8) Worker at station:

$ \hspace{10mm} Worker\_at\_Station_{worker,station}\geq Worker\_at\_Op_{worker,op}*Op\_at\_Station_{op,station}$

$ \hspace{10mm} \forall op\in Ops, worker\in Worker, station\in Stations$

$ \hspace{10mm} Worker\_at\_Station_{worker,station}\leq 1$

$ \hspace{10mm} \forall worker\in Worker, station\in Stations$

9)  Station Capacity fulfilled:

$ \hspace{10mm} \sum_{worker=1}^{Worker}Worker\_at\_Station_{worker,station} \leq Station\_Capacity_{station} \hspace{10mm} \forall station \in Stations$

10)  Actual processing time: 

$ \hspace{10mm} Actual\_Processing\_Time\_per\_Op_{op} \cdot \sum_{worker=1}^{Worker} Worker\_at\_Op_{worker, op} = (\sum_{worker=1}^{Worker} Worker\_at\_Op_{worker, op} \cdot Worker\_Multiplier_{worker, op}) \cdot Op\_Times_{op}$

$ \hspace{10mm} \forall op\in Ops$

11)  Each workers time occupied:

$ \hspace{10mm} Worker\_actual\_time\_Var_{worker} = \sum_{op=1}^{Operations} Worker\_at\_Op_{worker,op} \cdot Actual\_Processing\_Time\_per\_Op_{op}$
$ \hspace{10mm} \forall worker \in Worker$

12) Maximal occupied time of any worker

$ \hspace{10mm} Max\_Workhours\_Var = \max_{worker \in Worker} Worker\_actual\_time\_Var_{worker}$

---

#### Indices and sets
$op_1,op_2\in(1,..,N) \hspace{15mm}$  operations                      

$station\in(1,..,M) \hspace{17mm}$ stations                         

$(op_1,op_2)\in P \hspace{26mm}$ set of precedence relations     

$worker\in(1,..,W) \hspace{15mm}$ workers

$w\_type\in(1,..,WT) \hspace{20mm}$ worker type

$(lb,up) \in Worker\_Bounds\_per\_Op_{op_1} \hspace{20mm}$  Worker bounds per operation   

#### Parameters
$Op\_Times\_{op_1} \hspace{40mm}$  Processing time of the operation

$P \hspace{40mm}$  Set of precedence relations

$Worker\_Availability_{type} \hspace{20mm}$  Worker availability

$Worker\_Type\_Multiplier_{type,operation} \hspace{20mm}$  Worker type multiplier

$Worker\_Multiplier_{worker,operation} \hspace{20mm}$  Worker multiplier

$Station\_Capacity_{station} \hspace{20mm}$  Station capacity

$Worker\_Bounds\_per\_Op_{op_1} \hspace{20mm}$  Worker bounds per operation

#### Decision variables
$Op\_at\_Station_{op_1,station} \hspace{38mm}$  Binary variable             

$Cycle\_Time \hspace{41mm}$  Cycle time 

$Worker\_at\_Op_{worker,op_1} \hspace{41mm}$  Worker at operation

$Worker\_at\_Station_{worker,station} \hspace{41mm}$  Worker at station

$Actual\_Processing\_Time\_per\_Op_{op_1} \hspace{20mm}$  Actual processing time per operation

$Worker\_actual\_time\_Var_{worker} \hspace{20mm}$  Sum of time each worker works in each cycle

$Max\_Workhours\_Var \hspace{20mm}$  Maximum time any worker works in each cycle

------------------------------------------------------------

### Create the model
---

In [ ]:
# Model
m = gp.Model("MILP")

m.setParam("OutputFlag", 1)

### Sets and parameters
---

In [ ]:
WorkerCount = 0
for availability in worker_availability:
    WorkerCount += availability

Individual_Worker_Multiplier = []

MAX_INDIVIDUAL_MODIFIER = 0
for operation, worker in enumerate(worker_modifiers):
    for j, task in enumerate(worker):
        if task > MAX_INDIVIDUAL_MODIFIER and task != float("inf"):
            MAX_INDIVIDUAL_MODIFIER = task

for operation, worker in enumerate(worker_modifiers):
    for j, task in enumerate(worker):
        if task == float("inf"):
            worker_modifiers[operation][j] = 2 * MAX_INDIVIDUAL_MODIFIER


MAX_AMOUNT_MODIFIER = 0
for operation, modifiers in enumerate(worker_amount_modifiers):
    for amount, modifier in enumerate(modifiers):
        if modifier > MAX_AMOUNT_MODIFIER and modifier != float("inf"):
            MAX_AMOUNT_MODIFIER = modifier

for operation, modifiers in enumerate(worker_amount_modifiers):
    for amount, modifier in enumerate(modifiers):
        if modifier == float("inf"):
            worker_amount_modifiers[operation][amount] = 2 * MAX_AMOUNT_MODIFIER

w_type = 0
for availability in worker_availability:
    for operation in range(availability):
        bar_gap = []
        for modifier in worker_modifiers:
            bar_gap.append(modifier[w_type])
        Individual_Worker_Multiplier.append(bar_gap)
    w_type += 1

Insert the times needed for the single operations $t_i$ as an array.

In [ ]:
t = op_times

### Decision variables
---

Define the decision variables including the non-negativity and binary constraints:

**4) Binary decision variable**: 
The binary decision variable $S_{ik}$ is 1 if an operation i is assigned to station k, otherwise 0.

In [ ]:
Op_at_Station = m.addVars(OperationsLength, StationsLength, vtype=GRB.BINARY, name="Op_at_Station")

**5)  Non-negativity constraint**:  The cycle time C can't be negative.

In [ ]:
Cycle_Time = m.addVar(lb=0, name="Cycle_Time")

**5)  Auxiliary variable to break symmetry**:  Prefer assignment to lower station number in case of multiple solutions.

In [ ]:
symBreak_Op_at_Station = m.addVar(lb=0, name="symBreak_Op_at_Station")
symBreak_Worker_at_Op = m.addVar(lb=0, name="symBreak_Worker_at_Op")
symBreak_Worker_at_Station = m.addVar(lb=0, name="symBreak_Worker_at_Station")

**6) Binary decicion variable**: If operation op is assigned to worker.

In [ ]:
Worker_at_Op = m.addVars(WorkerCount, OperationsLength, vtype=GRB.BINARY, name="Worker_at_Operation")

**7) Binary decicion variable**: If a workers operations are at a station.

In [ ]:
Worker_at_Station = m.addVars(WorkerCount, StationsLength, vtype=GRB.BINARY, name="Worker_at_Station")

**8) An operations duration with the currently assigned workers**: Calculated by multiplying an operations time with the assigned workers average modifier. 

In [ ]:
Actual_Processing_Time_per_Op = m.addVars(OperationsLength, vtype=GRB.CONTINUOUS, name="Actual_Processing_Time_per_Op")

**9) Time a worker is occupied during a cycle**: For calculating the maximum over all workers. 

In [ ]:
Worker_actual_time_Var = m.addVars(WorkerCount, vtype=GRB.CONTINUOUS, name="Worker_actual_time_Var")

**10) Maximum over all workers occupied time**: For usage in objective. 

In [ ]:
Max_Workhours_Var = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Max_Workhours_Var")
Worker_Deviation = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Worker_Deviation")

**11) Number of workers used for an operation**

In [ ]:
nr_workers_at_op = m.addVars(WorkerCount + 1, OperationsLength, vtype=GRB.BINARY, name="nrWorkersAtOp")

**12) Amount modifier for a specific operation based on the assigned number of workers**

In [ ]:
actualAmountModifierOp = m.addVars(OperationsLength, vtype=GRB.CONTINUOUS, lb=0, ub=5, name="actualAmountModifierOp")

Only used for graphing the results.

In [ ]:
Theoretical_Processing_Time_per_Op = m.addVars(
    OperationsLength, vtype=GRB.CONTINUOUS, name="Theoretical_Processing_Time_per_Op"
)

m.addConstrs(
    (Theoretical_Processing_Time_per_Op[op] == op_times[op] for op in range(OperationsLength)),
    name="theoreticalProcessingTimeCon",
)

### Objective functions
---

Define the objective value to minimize the cycle time.

 1) The goal of the first objective is to minimize the cycle time:   $ \hspace{31mm} min \quad z = Cycle\_Time $

 2) The goal of the second objective is minimize the difference in time each worker is occupied during a cycle:   
 
 $ \hspace{10mm} min \quad z = \sum_{worker = 1}^{Worker} (Max\_Workhours\_Var - Worker\_actual\_time\_Var_{worker})$

In [ ]:
# Objective 1

m.setObjective(Cycle_Time + symBreak_Op_at_Station + symBreak_Worker_at_Op + symBreak_Worker_at_Station, GRB.MINIMIZE)

# Objective 2

# objective minimizes sum over differences from maximum
# m.setObjective(symBreak_Op_at_Station + symBreak_Worker_at_Op + symBreak_Worker_at_Station + Worker_Deviation
#    , GRB.MINIMIZE)

### Constraints
---


**1) Assign every operation only once**: Each operation may be assigned only once. The sum of the number of assignments of each operation across all stations must therefore add up to one. 

$ \qquad \sum_{k=1}^{M} S_{ik} = 1 \hspace{40mm} \forall i=1,...,N$

In [ ]:
m.addConstrs(
    (quicksum(Op_at_Station[i, k] for k in range(StationsLength)) == 1 for i in range(OperationsLength)),
    name="allOperCon",
)

**2) Complying the precedence relations**: An operation j may only begin when the predecessor operation i has been completed and can therefore not be processed on a station in front of i's processing station.

$ \qquad \sum_{k=1}^{M} S_{ik}\cdot k \leq \sum_{k=1}^{M} S_{jk}\cdot k \hspace{25mm}  \forall (i,j) \in P $

In [ ]:
m.addConstrs(
    (
        quicksum(Op_at_Station[i, k] * k for k in range(StationsLength))
        <= quicksum(Op_at_Station[j, k] * k for k in range(StationsLength))
        for i in range(OperationsLength)
        for j in range(OperationsLength)
        if [i + 1, j + 1] in PrecRelations
    ),
    name="precRelCon",
)

**3) Complying the cycle time**:
The cycle time must be smaller than the sum of the processing times of all operations carried out on a station k.

$ \qquad  \sum_{i=1}^NS_{ik} \cdot t_i \leq C \hspace{34mm} \forall k=1,...,M $




In [ ]:
m.addConstrs(
    (
        quicksum(Op_at_Station[op, station] * Actual_Processing_Time_per_Op[op] for op in range(OperationsLength))
        <= Cycle_Time
        for station in range(StationsLength)
    ),
    name="cycleTimeCon",
)

**4) Operation_at_Station is binary (done in definition)**

**5) Cycle_Time is larger or equal to zero (done in definition)**

**+ Symmetry breaking constraint**

In [ ]:
m.addConstr(
    symBreak_Op_at_Station
    >= quicksum(
        (Op_at_Station[i, k] * k * (0.1 ** (i + 1)) for k in range(StationsLength) for i in range(OperationsLength))
    ),
    name="symBreak_Op_at_StationCon",
)
m.addConstr(
    symBreak_Worker_at_Op
    >= quicksum(
        (
            Worker_at_Op[worker, op] * op * (0.1 ** (worker + 1))
            for op in range(OperationsLength)
            for worker in range(WorkerCount)
        )
    ),
    name="symBreak_Worker_at_OpCon",
)
m.addConstr(
    symBreak_Worker_at_Station
    >= quicksum(
        (
            Worker_at_Station[worker, station] * station * (0.1 ** (worker + 1))
            for station in range(StationsLength)
            for worker in range(WorkerCount)
        )
    ),
    name="symBreak_Worker_at_StationCon",
)

**6) Worker works at one station only**

In [ ]:
m.addConstrs(
    (
        quicksum(Worker_at_Station[worker, station] for station in range(StationsLength)) <= 1
        for worker in range(WorkerCount)
    ),
    name="workerOneStationCon",
)

**7) Operations bounds for number of assigned workers**

In [ ]:
m.addConstrs(
    (
        quicksum(Worker_at_Op[worker, op] for worker in range(WorkerCount)) <= worker_bounds[op][1]
        for op in range(OperationsLength)
    ),
    name="workerBoundsUpperCon",
)
m.addConstrs(
    (
        quicksum(Worker_at_Op[worker, op] for worker in range(WorkerCount)) >= worker_bounds[op][0]
        for op in range(OperationsLength)
    ),
    name="workerBoundsLowerCon",
)

**8) Worker at station (binary variable, calculated from the actual assignment)**

In [ ]:
m.addConstrs(
    (
        Worker_at_Station[w, s] >= Worker_at_Op[w, o] + Op_at_Station[o, s] - 1
        for w in range(WorkerCount)
        for o in range(OperationsLength)
        for s in range(StationsLength)
    ),
    name="workerAtStationLowerCon",
)

**9) Stations capacity bounds for number of assigned workers**

In [ ]:
m.addConstrs(
    (
        quicksum(Worker_at_Station[worker, station] for worker in range(WorkerCount)) <= station_capacity[station]
        for station in range(StationsLength)
    ),
    name="stationCapacityCon",
)

**10) Calculate the worker amount modifier for every operation**

In [ ]:
m.addConstrs(
    (
        ((quicksum(nr_workers_at_op[worker, op] for worker in range(1, WorkerCount + 1))) == 1)
        for op in range(OperationsLength)
    ),
    name="exactlyOneWorkerNumberPerOpCon",
)

m.addConstrs(
    (
        (quicksum(nr_workers_at_op[i, op] * i for i in range(1, WorkerCount + 1)))
        == quicksum(Worker_at_Op[worker, op] for worker in range(WorkerCount))
        for op in range(OperationsLength)
    ),
    name="nrWorkersAtOpCon",
)

m.addConstrs(
    (
        actualAmountModifierOp[op]
        == quicksum(
            nr_workers_at_op[worker, op] * worker_amount_modifiers[op][worker - 1]
            for worker in range(1, WorkerCount + 1)
        )
        for op in range(OperationsLength)
    ),
    name="actualAmountModifierOpCon",
)

**11) Set Actual processing time variables**

In [ ]:
m.addConstrs(
    (
        Actual_Processing_Time_per_Op[op] * quicksum(Worker_at_Op[w, op] for w in range(WorkerCount))
        == quicksum(
            Worker_at_Op[worker, op] * Individual_Worker_Multiplier[worker][op] for worker in range(WorkerCount)
        )
        * op_times[op]
        * actualAmountModifierOp[op]
        for op in range(OperationsLength)
    ),
    name="actualProcessingTimeCon",
)

**12) and 13) For second objective: Set workers individual worktime per cycle and find maximum**

In [ ]:
m.addConstrs(
    (
        Worker_actual_time_Var[worker]
        == quicksum(Worker_at_Op[worker, op] * Actual_Processing_Time_per_Op[op] for op in range(OperationsLength))
        for worker in range(WorkerCount)
    ),
    name="Worker_actual_time_Constr",
)

m.addConstrs(
    (Max_Workhours_Var >= Worker_actual_time_Var[worker] for worker in range(WorkerCount)), name="Max_Workhours_Constr"
)
m.addConstr(
    (Worker_Deviation >= quicksum(Max_Workhours_Var - Worker_actual_time_Var[worker] for worker in range(WorkerCount))),
    name="WorkerDeviationCon",
)

**14) Ensure only valid assignments for workers to operations**

In [ ]:
# constraint: Worker_at_Op[worker, op] * Individual_Worker_Multiplier[worker][op] < INFINITY_MULTIPLIER for all worker, op
m.addConstrs(
    (
        Worker_at_Op[worker, op] * Individual_Worker_Multiplier[worker][op] <= MAX_INDIVIDUAL_MODIFIER
        for worker in range(WorkerCount)
        for op in range(OperationsLength)
    ),
    name="workerAssignmentInfinityCon",
)
# constraint: actualAmountModifierOp[op] < INFINITY_MULTIPLIER
m.addConstrs(
    (actualAmountModifierOp[op] <= MAX_AMOUNT_MODIFIER for op in range(OperationsLength)),
    name="actualAmountModifierOpInfinityCon",
)

### Solve and display results
---

Get the whole model m displayed.

In [ ]:
m.update()
m

Run the model m.

In [ ]:
m.setParam("TimeLimit", 120)
m.setParam("MIPFocus", 3)
m.setParam("MIPgap", 0.01)

m.optimize()

### Get the results displayed.

Print out the objective value C as well as the termination status and the primal status. Used for debugging and quick overview of the results.  
**For complete visualization, look at `alb-2-visualization.ipynb`** 

In [ ]:
# Print results
if m.status == GRB.OPTIMAL or m.status == GRB.TIME_LIMIT:
    print("Objective value: ", m.objVal)
    print("Cycle Time C: ", Cycle_Time.x)
    print("Solution:")
    for operation in range(OperationsLength):
        for station in range(StationsLength):
            if Op_at_Station[operation, station].x > 0.5:
                print("Operation ", operation + 1, " is assigned to station ", station + 1)
else:
    print("No solution found")
    if m.status == GRB.INFEASIBLE:
        print("Model is infeasible")
        m.computeIIS()
        m.write("model.ilp")
        # Print the constraints that are causing the infeasibility
        for c in m.getConstrs():
            if c.IISConstr:
                print(f"Infeasible constraint: {c.constrName}")
    elif m.status == GRB.UNBOUNDED:
        print("Model is unbounded")
    elif m.status == GRB.INF_OR_UNBD:
        print("Model is infeasible or unbounded")
    else:
        print("Optimization was stopped with status", m.status)

m.printStats()
for w, op in Worker_at_Op:
    if Worker_at_Op[w, op].x:
        if Individual_Worker_Multiplier[w][op] > MAX_INDIVIDUAL_MODIFIER:
            print("ERROR: Bad assignment, worker cannot perform the following operation")
        print("Worker ", w + 1, " is assigned to operation ", op + 1)

for w, station in Worker_at_Station:
    if Worker_at_Station[w, station].x:
        print("Worker ", w + 1, " is assigned to station ", station + 1)

for op in range(OperationsLength):
    print("Operation ", op + 1, " has normal processing time ", op_times[op])

for op in range(OperationsLength):
    print("Operation ", op + 1, " has actual processing time ", Actual_Processing_Time_per_Op[op].x)

for op in range(OperationsLength):
    print("Operation ", op + 1, " has worker amount modifier ", actualAmountModifierOp[op].x)

**Worker time, grouped by stations**

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(10, 8))

bar_width = 0.05
bar_gap = 0.08
group_gap = -0.4

group_multiplier = 0
stations_x_ticks = []
total_sum_workers = []
colors = plt.get_cmap("tab20c").colors
color_index = 0

for station in range(StationsLength):
    workers_sum = []
    worker_labels = []
    bar_positions = []
    for worker in range(WorkerCount):
        worker_station_sum = 0
        if Worker_at_Station[worker, station].x:
            for op in range(OperationsLength):
                worker_station_sum += Worker_at_Op[worker, op].x * Actual_Processing_Time_per_Op[op].x
            workers_sum.append(worker_station_sum)
            worker_labels.append(f"W {worker + 1}")
    for operation in range(len(workers_sum)):
        bar_positions.append(
            group_multiplier * group_gap + station + (operation - (len(workers_sum) - 1) / 2) * bar_gap
        )
        total_sum_workers.append(workers_sum[operation])

    stations_x_ticks.append(group_multiplier * group_gap + station)
    group_multiplier += 1

    bars = axs.bar(
        bar_positions, workers_sum, bar_width, label=f"Station {station + 1}", color=colors[color_index % len(colors)]
    )
    color_index += 1
    axs.bar_label(bars, labels=worker_labels, label_type="edge")

# Plot average time per worker as an hline
average_time = np.average(total_sum_workers)
axs.axhline(y=average_time, color="#af2020", linestyle="dotted", label=f"Average Worker Time (t={average_time:.2f})")

axs.set_xticks(stations_x_ticks)
axs.set_xticklabels([f"Station {i + 1}" for i in range(StationsLength)])

axs.set_title("Individual Worker Time per Station")
axs.set_xlabel("Stations")
axs.set_ylabel("Processing Time")
axs.legend(bbox_to_anchor=(1, 1))

plt.show()


**Time per station and operation**

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(10, 8))

# Stacked bar chart for visualizing the time at each station, this will nicely show the cycle time
colors = plt.get_cmap("tab20c").colors
color_index = 0

for station in range(0, StationsLength):
    station_sum = 0
    for operation in range(0, OperationsLength):
        if Op_at_Station[operation, station].x:
            bar = axs.bar(
                station + 1,
                Actual_Processing_Time_per_Op[operation].x,
                bottom=station_sum,
                label=f"Operation {operation + 1}",
                edgecolor="black",
                color=colors[color_index % len(colors)],
            )
            axs.bar_label(bar)
            station_sum += Actual_Processing_Time_per_Op[operation].x
            color_index += 1


# Draw horizontal line at cycle time
axs.axhline(y=Cycle_Time.x, color="g", linestyle="--", label=f"Cycle Time (t={Cycle_Time.x:.2f})")

# Place legend outside of plot
# Sort the legend labels alphabetically, credit: https://stackoverflow.com/a/27512450/5541326
handles, labels = axs.get_legend_handles_labels()
# sort both labels and handles by labels
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
axs.legend(handles, labels, bbox_to_anchor=(1, 1), loc="upper left")

axs.set_xticks(range(1, StationsLength + 1))
axs.set_xlabel("Stations")
axs.set_ylabel("Total Time")
axs.set_title("Operations performed at station with cumulative processing time")

plt.show()


**Time per station with assigned workers**

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(12, 4))

# Horizontal bar chart for visualizing the number of workers at each station
colors = plt.get_cmap("tab20c").colors
color_index = 0
max_workers = 0

for station in range(0, StationsLength):
    workers = 0
    for operation in range(0, WorkerCount):
        if Worker_at_Station[operation, station].x:
            workers += 1
    bar = axs.barh(
        station + 1, workers, height=0.3, label=f"Station {station + 1}", color=colors[color_index % len(colors)]
    )
    if max_workers < workers:
        max_workers = workers
    color_index += 1

axs.set_yticks(range(1, StationsLength + 1))
axs.set_yticklabels([f"Station {i + 1}" for i in range(StationsLength)])
axs.set_ylabel("Stations")
axs.set_xlabel("Number of Workers")
axs.set_xticks(range(0, max_workers + 1))
axs.set_title("Number of Workers per Station")

plt.show()


**Number of tasks per worker**

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(12, 4))

# Horizontal bar chart for visualizing the number of tasks for each worker
colors = plt.get_cmap("tab20c").colors
color_index = 0

max_tasks = 0

for worker in range(0, WorkerCount):
    tasks = 0
    for operation in range(0, OperationsLength):
        if Worker_at_Op[worker, operation].x:
            tasks += 1
    bar = axs.barh(worker + 1, tasks, height=0.3, label=f"Worker {worker + 1}", color=colors[color_index % len(colors)])
    if max_tasks < tasks:
        max_tasks = tasks
    color_index += 1

axs.set_yticks(range(1, WorkerCount + 1))
axs.set_yticklabels([f"Worker {i + 1}" for i in range(WorkerCount)])
axs.set_ylabel("Workers")
axs.set_xlabel("Number of Tasks")
axs.set_xticks(range(0, max_tasks + 1))
axs.set_title("Number of Tasks per Worker")

plt.show()

# Pareto front

In [ ]:
def create_pareto_front(primary_obj_selection, steps, resolution_factor=1_000, output_file=None, time_limit=600):
    model_output = {
        "Cycle_Time": [],
        "Fairness": [],
        "symBreak_Op_at_Station": [],
        "symBreak_Worker_at_Op": [],
        "symBreak_Worker_at_Station": [],
        "Max_Workhours_Var": [],
        "Op_at_Station": [],
        "Worker_at_Operation": [],
        "Worker_at_Station": [],
        "Actual_Processing_Time_per_Op": [],
        "Worker_actual_time_Var": [],
        "Worker_Deviation": [],
        "Actual_Amount_Modifier_Op": [],
        "Theoretical_Processing_Time_per_Op": [],
    }
    m.setParam("OutputFlag", 0) 
    m.setParam("TimeLimit", time_limit)
    # Default file path
    if output_file is None:
        from datetime import datetime

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = f"{steps}_steps_pareto_front_{primary_obj_selection}_{timestamp}.json"

    cycle_time_obj = Cycle_Time + symBreak_Op_at_Station + symBreak_Worker_at_Op + symBreak_Worker_at_Station
    fairness_obj = Worker_Deviation + symBreak_Op_at_Station + symBreak_Worker_at_Op + symBreak_Worker_at_Station
    # set upper bound for symBreaks in testing symbreaks where always close to 0
    bound = 10
    symBreak_Op_at_Station.UB = bound
    symBreak_Worker_at_Op.UB = bound
    symBreak_Worker_at_Station.UB = bound
    if primary_obj_selection == "Cycle_Time":
        secondary_objective = fairness_obj
        primary_objective = cycle_time_obj
    elif primary_obj_selection == "Fairness":
        secondary_objective = cycle_time_obj
        primary_objective = fairness_obj
        Cycle_Time.UB = sum(op_times)
    # LB
    m.setObjective(secondary_objective, GRB.MINIMIZE)
    m.optimize()
    # UB
    if primary_obj_selection == "Cycle_Time":
        secondary_obj_min = Worker_Deviation.X if m.status == GRB.OPTIMAL or m.status == GRB.TIME_LIMIT else None
        m.setObjective(Cycle_Time, GRB.MINIMIZE)
        m.optimize()
        secondary_obj_max = Worker_Deviation.X if m.status == GRB.OPTIMAL or m.status == GRB.TIME_LIMIT else None
    elif primary_obj_selection == "Fairness":
        secondary_obj_min = Cycle_Time.X if m.status == GRB.OPTIMAL or m.status == GRB.TIME_LIMIT else None
        m.setObjective(Worker_Deviation, GRB.MINIMIZE)
        m.optimize()
        secondary_obj_max = Cycle_Time.X if m.status == GRB.OPTIMAL or m.status == GRB.TIME_LIMIT else None
    if secondary_obj_min is None:
        raise RuntimeError("Could not find minimum value of secondary objective")
    if secondary_obj_max is None:
        raise RuntimeError("Could not find maximum value of secondary objective")
    epsilon_values = (
        secondary_obj_min
        + (secondary_obj_max - secondary_obj_min)
        * (np.logspace(0, 1, steps, base=resolution_factor) - 1)
        / (resolution_factor - 1)
    ).tolist()
    # useless in most cases but very high resolution factor / base can lead to extreme values-> floating point errors so this ensures that the last value is the actual max value
    epsilon_values = epsilon_values[:-1] + [secondary_obj_max]
    # initialize worst found with infinity
    if primary_obj_selection == "Cycle_Time":
        worst_cycle_time = np.inf
    elif primary_obj_selection == "Fairness":
        worst_worker_deviation = np.inf

    pareto_front = []
    current_step = 0
    model_output["secondary_obj_min"] = secondary_obj_min
    model_output["secondary_obj_max"] = secondary_obj_max
    model_output["worker_bounds"] = worker_bounds
    model_output["worker_availability"] = worker_availability
    model_output["worker_modifiers"] = worker_modifiers
    model_output["individual_worker_multiplier"] = Individual_Worker_Multiplier
    model_output["worker_amount_modifiers"] = worker_amount_modifiers
    model_output["station_capacity"] = station_capacity
    model_output["precedence_relations"] = PrecRelations
    model_output["station_count"] = StationsLength
    model_output["task_count"] = OperationsLength
    model_output["task_times"] = op_times
    model_output["worker_count"] = WorkerCount
    model_output["model_name"] = file_path
    model_output["nr_steps"] = steps
    model_output["max_amount_modifier"] = MAX_AMOUNT_MODIFIER
    model_output["max_individual_modifier"] = MAX_INDIVIDUAL_MODIFIER
    for epsilon in epsilon_values:
        m.setObjective(primary_objective, GRB.MINIMIZE)
        
        if primary_obj_selection == "Cycle_Time":
            Cycle_Time.UB = worst_cycle_time
            Worker_Deviation.UB = epsilon
        elif primary_obj_selection == "Fairness":
            Cycle_Time.UB = epsilon
            Worker_Deviation.UB = worst_worker_deviation

        m.optimize()
        if m.status == GRB.OPTIMAL or m.status == GRB.TIME_LIMIT:
            pareto_front.append((Cycle_Time.X, Worker_Deviation.X, epsilon, current_step))
            model_output["Cycle_Time"].append(Cycle_Time.X)
            model_output["Fairness"].append(Worker_Deviation.X)
            model_output["symBreak_Op_at_Station"].append(symBreak_Op_at_Station.X)
            model_output["symBreak_Worker_at_Op"].append(symBreak_Worker_at_Op.X)
            model_output["symBreak_Worker_at_Station"].append(symBreak_Worker_at_Station.X)
            model_output["Max_Workhours_Var"].append(Max_Workhours_Var.X)
            model_output["Op_at_Station"].append({str(k): v for k, v in m.getAttr("X", Op_at_Station).items()})
            model_output["Worker_at_Operation"].append({str(k): v for k, v in m.getAttr("X", Worker_at_Op).items()})
            model_output["Worker_at_Station"].append({str(k): v for k, v in m.getAttr("X", Worker_at_Station).items()})
            model_output["Actual_Processing_Time_per_Op"].append(
                {str(k): v for k, v in m.getAttr("X", Actual_Processing_Time_per_Op).items()}
            )
            model_output["Theoretical_Processing_Time_per_Op"].append(
                {str(k): v for k, v in m.getAttr("X", Theoretical_Processing_Time_per_Op).items()}
            )
            model_output["Worker_actual_time_Var"].append(
                {str(k): v for k, v in m.getAttr("X", Worker_actual_time_Var).items()}
            )
            model_output["Worker_Deviation"].append(Worker_Deviation.X)
            model_output["Actual_Amount_Modifier_Op"].append(
                {str(k): v for k, v in m.getAttr("X", actualAmountModifierOp).items()}
            )
            worst_worker_deviation = Worker_Deviation.X
            worst_cycle_time = Cycle_Time.X
        else:
            pareto_front.append((None, None, epsilon, current_step))
        print("--------------------------------------------------------------------------------")
        print(f"step: {current_step}")
        print(f"Cycle Time: {model_output['Cycle_Time'][-1]}")
        print(f"fairness: {model_output['Fairness'][-1]}")
        print(f"New fairness must be smaller than epsilon: {epsilon}")
        print(
            f"Status: {m.status} ({'optimal' if m.status == GRB.OPTIMAL else 'time limit' if m.status == GRB.TIME_LIMIT else 'other'})"
        )
        print("--------------------------------------------------------------------------------")
        current_step += 1

    model_output["pareto_front"] = pareto_front

    try:
        if not os.path.exists("output"):
            os.makedirs("output")
        with open("output/" + output_file, "w") as f:
            json.dump(model_output, f)
        print(f"Results saved to {output_file}")
    except Exception as e:
        print(f"Error saving results: {e}")

    return pareto_front, model_output


pareto_front, m_out = create_pareto_front("Cycle_Time", 10, time_limit=120)


# Vizualization of the results
For each step in the pareto front, get the results. Useful for debugging and checking the results.  
**For complete visualization, look at `alb-2-visualization.ipynb`** 

In [ ]:
# Draw graph at which step-> (keep in mind different steps/epsilon constraints often result in the same point)
pstep = 1

In [ ]:
# Print results
print("Cycle Time C: ", m_out["Cycle_Time"][pstep])
print("Fairness: ", m_out["Worker_Deviation"][pstep])
print("Solution:")
for operation in range(OperationsLength):
    for station in range(StationsLength):
        converted_op_at_station = {eval(k): v for k, v in m_out["Op_at_Station"][pstep].items()}
        if converted_op_at_station[operation, station] > 0.5:
            print("Operation ", operation + 1, " is assigned to station ", station + 1)

converted_worker_at_op = {eval(k): v for k, v in m_out["Worker_at_Operation"][pstep].items()}
for w, op in converted_worker_at_op:
    if converted_worker_at_op[w, op]:
        if Individual_Worker_Multiplier[w][op] > 5:
            print("ERROR: Bad assignment, worker cannot perform the following operation")
        print("Worker ", w + 1, " is assigned to operation ", op + 1)

converted_worker_at_station = {eval(k): v for k, v in m_out["Worker_at_Station"][pstep].items()}
for w, station in converted_worker_at_station:
    if converted_worker_at_station[w, station]:
        print("Worker ", w + 1, " is assigned to station ", station + 1)

for op in range(OperationsLength):
    print("Operation ", op + 1, " has normal processing time ", op_times[op])

converted_actual_processing_time_per_op = {eval(k): v for k, v in m_out["Actual_Processing_Time_per_Op"][pstep].items()}
for op in range(OperationsLength):
    print("Operation ", op + 1, " has actual processing time ", converted_actual_processing_time_per_op[op])

converted_actual_amount_modifier_op = {eval(k): v for k, v in m_out["Actual_Amount_Modifier_Op"][pstep].items()}
for op in range(OperationsLength):
    print("Operation ", op + 1, " has worker amount modifier ", converted_actual_amount_modifier_op[op])

**Worker time, grouped by stations**

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(10, 8))

# Stacked bar chart for visualizing the time at each station, this will nicely show the cycle time
colors = plt.get_cmap("tab20c").colors
color_index = 0

for station in range(0, StationsLength):
    station_sum = 0
    for operation in range(0, OperationsLength):
        if converted_op_at_station[operation, station]:
            bar = axs.bar(
                station + 1,
                converted_actual_processing_time_per_op[operation],
                bottom=station_sum,
                label=f"Operation {operation + 1}",
                edgecolor="black",
                color=colors[color_index % len(colors)],
            )
            axs.bar_label(bar)
            station_sum += converted_actual_processing_time_per_op[operation]
            color_index += 1


# Draw horizontal line at cycle time
axs.axhline(
    y=m_out["Cycle_Time"][pstep],
    color="g",
    linestyle="--",
    label=f"Cycle Time (t={m_out['Cycle_Time'][pstep]:.2f})",
)

# Place legend outside of plot
# Sort the legend labels alphabetically, credit: https://stackoverflow.com/a/27512450/5541326
handles, labels = axs.get_legend_handles_labels()
# sort both labels and handles by labels
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
axs.legend(handles, labels, bbox_to_anchor=(1, 1), loc="upper left")

axs.set_xticks(range(1, StationsLength + 1))
axs.set_xlabel("Stations")
axs.set_ylabel("Total Time")
axs.set_title("Operations performed at station with cumulative processing time")

plt.show()


**Time per station and operation**

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(10, 8))

bar_width = 0.05
bar_gap = 0.08
group_gap = -0.4

group_multiplier = 0
stations_x_ticks = []
total_sum_workers = []
colors = plt.get_cmap("tab20c").colors
color_index = 0

for station in range(StationsLength):
    workers_sum = []
    worker_labels = []
    bar_positions = []
    for worker in range(WorkerCount):
        worker_station_sum = 0
        if converted_worker_at_station[worker, station]:
            for op in range(OperationsLength):
                worker_station_sum += converted_worker_at_op[worker, op] * converted_actual_processing_time_per_op[op]
            workers_sum.append(worker_station_sum)
            worker_labels.append(f"W {worker + 1}")
    for operation in range(len(workers_sum)):
        bar_positions.append(
            group_multiplier * group_gap + station + (operation - (len(workers_sum) - 1) / 2) * bar_gap
        )
        total_sum_workers.append(workers_sum[operation])

    stations_x_ticks.append(group_multiplier * group_gap + station)
    group_multiplier += 1

    bars = axs.bar(
        bar_positions,
        workers_sum,
        bar_width,
        label=f"Station {station + 1}",
        color=colors[color_index % len(colors)],
    )
    color_index += 1
    axs.bar_label(bars, labels=worker_labels, label_type="edge")

# Plot average time per worker as an hline
average_time = np.average(total_sum_workers)
axs.axhline(
    y=average_time,
    color="#af2020",
    linestyle="dotted",
    label=f"Average Worker Time (t={average_time:.2f})",
)

axs.set_xticks(stations_x_ticks)
axs.set_xticklabels([f"Station {i + 1}" for i in range(StationsLength)])

axs.set_title("Individual Worker Time per Station")
axs.set_xlabel("Stations")
axs.set_ylabel("Processing Time")
axs.legend(bbox_to_anchor=(1, 1))

plt.show()


**Time per station with assigned workers**

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(12, 4))

# Horizontal bar chart for visualizing the number of workers at each station
colors = plt.get_cmap("tab20c").colors
color_index = 0
max_workers = 0

for station in range(0, StationsLength):
    workers = 0
    for operation in range(0, WorkerCount):
        if converted_worker_at_station[operation, station]:
            workers += 1
    bar = axs.barh(
        station + 1,
        workers,
        height=0.3,
        label=f"Station {station + 1}",
        color=colors[color_index % len(colors)],
    )
    if max_workers < workers:
        max_workers = workers
    color_index += 1

axs.set_yticks(range(1, StationsLength + 1))
axs.set_yticklabels([f"Station {i + 1}" for i in range(StationsLength)])
axs.set_ylabel("Stations")
axs.set_xlabel("Number of Workers")
axs.set_xticks(range(0, max_workers + 1))
axs.set_title("Number of Workers per Station")

plt.show()


**Number of tasks per worker**

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(12, 4))

# Horizontal bar chart for visualizing the number of tasks for each worker
colors = plt.get_cmap("tab20c").colors
color_index = 0

max_tasks = 0

for worker in range(0, WorkerCount):
    tasks = 0
    for operation in range(0, OperationsLength):
        if converted_worker_at_op[worker, operation]:
            tasks += 1
    bar = axs.barh(
        worker + 1,
        tasks,
        height=0.3,
        label=f"Worker {worker + 1}",
        color=colors[color_index % len(colors)],
    )
    if max_tasks < tasks:
        max_tasks = tasks
    color_index += 1

axs.set_yticks(range(1, WorkerCount + 1))
axs.set_yticklabels([f"Worker {i + 1}" for i in range(WorkerCount)])
axs.set_ylabel("Workers")
axs.set_xlabel("Number of Tasks")
axs.set_xticks(range(0, max_tasks + 1))
axs.set_title("Number of Tasks per Worker")

plt.show()